<a href="https://colab.research.google.com/github/HJK-G/AI/blob/master/CentAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DESCRIPTION
This is an AI project to play a game.

It uses Deep Reinforcement Learning (DQNAgent)

https://keon.io/deep-q-learning/

The game is Centakill, made by one of my friends, Kaiway Tang.

Go check them out at: https://kaiway.itch.io/

Or just the game at: https://kaiway.itch.io/centa



imports:

In [1]:
from collections import deque
import random
import numpy as np
import time

from keras.models import Sequential, model_from_json
from keras.layers import Dense
from keras.optimizers import Adam

from mss import mss
from PIL import Image
import cv2

import pyautogui
import pytesseract

Using TensorFlow backend.


ModuleNotFoundError: ignored

This is DQNAgent. It was taken from https://keon.io/deep-q-learning/

This is a class that contains the model.

The values to create the model are modifiable.



In [0]:


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=50000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, min(batch_size, len(self.memory)))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * \
                         np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save(self):
        self.model.save_weights('model.h5')

    def load(self):
        self.model.load_weights('model.h5')
